In [ ]:
import math
import numpy as np
import tensorflow as tf
from __future__ import division, print_function, unicode_literals
from tensorflow.examples.tutorials.mnist import input_data as mnist_data

# to make this notebook's output stable across runs
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

Below, we create a CNN network for the MNIST dataset with the following architecture:
* **Conv. layer 1:** computes 32 feature maps using a 5x5 filter with ReLU activation.
* **Pooling layer 1:** max pooling layer with a 2x2 filter and stride of 2.
* **Conv. layer 2:** computes 64 feature maps using a 5x5 filter.
* **Pooling layer 2:** max pooling layer with a 2x2 filter and stride of 2.
* **Dense layer:** densely connected layer with 1024 neurons.
* **Logits layer**

First we make the network manually.

In [ ]:
# manual building layers
reset_graph()

n_epochs = 1000
batch_size = 100

########################################
# load the mnist data
########################################
mnist = mnist_data.read_data_sets("data", one_hot=True, reshape=False, validation_size=0)

########################################
# define placeholders
########################################
X = tf.placeholder(tf.float32, [None, 28, 28, 1])
y_true = tf.placeholder(tf.float32, [None, 10])
pkeep = tf.placeholder(tf.float32)

########################################
# build the model
########################################
# Convolutional Layer #1
# Computes 32 feature maps using a 5x5 filter with ReLU activation.
# Padding is added to preserve width and height.
# Input Tensor Shape: [batch_size, 28, 28, 1]
# Output Tensor Shape: [batch_size, 28, 28, 32]
W1 = tf.Variable(tf.truncated_normal([5, 5, 1, 32], stddev=0.1))  # 5x5 filter, 1 input channel, 32 output channels
B1 = tf.Variable(tf.constant(0.1, tf.float32, [32]))
stride = 1  # output is 28x28
conv1 = tf.nn.relu(tf.nn.conv2d(X, W1, strides=[1, stride, stride, 1], padding="SAME") + B1)

# Pooling Layer #1
# First max pooling layer with a 2x2 filter and stride of 2
# Input Tensor Shape: [batch_size, 28, 28, 32]
# Output Tensor Shape: [batch_size, 14, 14, 32]
pool1 = tf.nn.max_pool(conv1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="SAME")

# Convolutional Layer #2
# Computes 64 features using a 5x5 filter.
# Padding is added to preserve width and height.
# Input Tensor Shape: [batch_size, 14, 14, 32]
# Output Tensor Shape: [batch_size, 14, 14, 64]
W2 = tf.Variable(tf.truncated_normal([5, 5, 32, 64], stddev=0.1))  # 5x5 filter, 32 input channel, 64 output channels
B2 = tf.Variable(tf.constant(0.1, tf.float32, [64]))
stride = 1  # output is 28x28
conv2 = tf.nn.relu(tf.nn.conv2d(pool1, W2, strides=[1, stride, stride, 1], padding="SAME") + B2)

# Pooling Layer #2
# Second max pooling layer with a 2x2 filter and stride of 2
# Input Tensor Shape: [batch_size, 14, 14, 64]
# Output Tensor Shape: [batch_size, 7, 7, 64]
pool2 = tf.nn.max_pool(conv2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="SAME")

# Flatten tensor into a batch of vectors
# Input Tensor Shape: [batch_size, 7, 7, 64]
# Output Tensor Shape: [batch_size, 7 * 7 * 64]
pool2_flat = tf.reshape(pool2, [-1, 7 * 7 * 64])

# Dense Layer
# Densely connected layer with 1024 neurons
# Input Tensor Shape: [batch_size, 7 * 7 * 64]
# Output Tensor Shape: [batch_size, 1024]
W3 = tf.Variable(tf.truncated_normal([7 * 7 * 64, 1024], stddev=0.1))  # 7x7x64 feature, 1024 neurons
B3 = tf.Variable(tf.constant(0.1, tf.float32, [1024]))
dense = tf.nn.relu(tf.matmul(pool2_flat, W3) + B3)

# Add dropout operation
dropout = tf.nn.dropout(dense, pkeep)

# Logits layer
# Input Tensor Shape: [batch_size, 1024]
# Output Tensor Shape: [batch_size, 10]
W4 = tf.Variable(tf.truncated_normal([1024, 10], stddev=0.1))  # 1024 feature, 10 neurons
B4 = tf.Variable(tf.constant(0.1, tf.float32, [10]))
logits = tf.matmul(dropout, W4) + B4
y_hat = tf.nn.softmax(logits)

########################################
# define the cost and accuracy functions
########################################
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y_true)
cross_entropy = tf.reduce_mean(cross_entropy) * 100

correct_prediction = tf.equal(tf.argmax(y_hat, 1), tf.argmax(y_true, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

########################################
# define the optimizer
########################################
lr = 0.003
optimizer = tf.train.AdamOptimizer(lr)
train_step = optimizer.minimize(cross_entropy)

########################################
# execute the model
########################################
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)

    for i in range(n_epochs):
        # load batch of images and correct answers
        batch_X, batch_y = mnist.train.next_batch(batch_size)
        
        # learning rate decay
        if i % 100 == 0:
            a, c = sess.run([accuracy, cross_entropy], feed_dict={X: batch_X, y_true: batch_y, pkeep: 1.0})
            print('epoch {}: accurecy = {}, loss = {}'.format(i, a, c))

        # train
        sess.run(train_step, feed_dict={X: batch_X, y_true: batch_y, pkeep: 0.75})
        
    a, c = sess.run([accuracy, cross_entropy], feed_dict={X: mnist.test.images, y_true: mnist.test.labels, pkeep: 1.0})
    print('test data: accurecy = {}, loss = {}'.format(a, c))  

Now, we take advantage of the `tf.layers` to build the network.

In [ ]:
reset_graph()

n_epochs = 1000
batch_size = 100

########################################
# load the mnist data
########################################
mnist = mnist_data.read_data_sets("data", one_hot=True, reshape=False, validation_size=0)

#######################################
# defineplaceholders
########################################
X = tf.placeholder(tf.float32, [None, 28, 28, 1])
y_true = tf.placeholder(tf.float32, [None, 10])
pkeep = tf.placeholder(tf.float32)

########################################
# build the model
########################################
# Convolutional Layer #1
# Computes 32 feature maps using a 5x5 filter with ReLU activation.
# Padding is added to preserve width and height.
# Input Tensor Shape: [batch_size, 28, 28, 1]
# Output Tensor Shape: [batch_size, 28, 28, 32]
conv1 = tf.layers.conv2d(inputs=X, filters=32, kernel_size=[5, 5], padding="same", activation=tf.nn.relu)

# Pooling Layer #1
# First max pooling layer with a 2x2 filter and stride of 2
# Input Tensor Shape: [batch_size, 28, 28, 32]
# Output Tensor Shape: [batch_size, 14, 14, 32]
pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)

# Convolutional Layer #2
# Computes 64 features using a 5x5 filter.
# Padding is added to preserve width and height.
# Input Tensor Shape: [batch_size, 14, 14, 32]
# Output Tensor Shape: [batch_size, 14, 14, 64]
conv2 = tf.layers.conv2d(inputs=pool1, filters=64, kernel_size=[5, 5], padding="same", activation=tf.nn.relu)

# Pooling Layer #2
# Second max pooling layer with a 2x2 filter and stride of 2
# Input Tensor Shape: [batch_size, 14, 14, 64]
# Output Tensor Shape: [batch_size, 7, 7, 64]
pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)

# Flatten tensor into a batch of vectors
# Input Tensor Shape: [batch_size, 7, 7, 64]
# Output Tensor Shape: [batch_size, 7 * 7 * 64]
pool2_flat = tf.reshape(pool2, [-1, 7 * 7 * 64])

# Dense Layer
# Densely connected layer with 1024 neurons
# Input Tensor Shape: [batch_size, 7 * 7 * 64]
# Output Tensor Shape: [batch_size, 1024]
dense = tf.layers.dense(inputs=pool2_flat, units=1024, activation=tf.nn.relu)

# Add dropout operation
dropout = tf.layers.dropout(inputs=dense, rate=pkeep)

# Logits layer
# Input Tensor Shape: [batch_size, 1024]
# Output Tensor Shape: [batch_size, 10]
logits = tf.layers.dense(inputs=dropout, units=10)
y_hat = tf.nn.softmax(logits)

########################################
# define the cost and accuracy functions
########################################
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y_true)
cross_entropy = tf.reduce_mean(cross_entropy) * 100

correct_prediction = tf.equal(tf.argmax(y_hat, 1), tf.argmax(y_true, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

########################################
# define the optimizer
########################################
lr = 0.003
optimizer = tf.train.AdamOptimizer(lr)
train_step = optimizer.minimize(cross_entropy)

########################################
# execute the model
########################################
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)

    for i in range(n_epochs):
        # load batch of images and correct answers
        batch_X, batch_y = mnist.train.next_batch(batch_size)
        
        # learning rate decay
        if i % 100 == 0:
            a, c = sess.run([accuracy, cross_entropy], feed_dict={X: batch_X, y_true: batch_y, pkeep: 1.0})
            print('epoch {}: accurecy = {}, loss = {}'.format(i, a, c))

        # train
        sess.run(train_step, feed_dict={X: batch_X, y_true: batch_y, pkeep: 0.75})
        
    a, c = sess.run([accuracy, cross_entropy], feed_dict={X: mnist.test.images, y_true: mnist.test.labels, pkeep: 1.0})
    print('test data: accurecy = {}, loss = {}'.format(a, c))  

Finally, we use Keras to build the above network.

In [ ]:
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D

reset_graph()

n_epochs = 1000
batch_size = 100

########################################
# load the mnist data
########################################
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)

# Rscaling of the pixel values from 0 to 255 to the range between 0 and 1. It improves the learning speed.
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

########################################
# buid the model
########################################
model = Sequential()
model.add(Conv2D(32, kernel_size=[5, 5], activation='relu', input_shape=[28, 28, 1]))
model.add(MaxPooling2D(pool_size=[2, 2], strides=2))
model.add(Conv2D(64, kernel_size=[5, 5], activation='relu'))
model.add(MaxPooling2D(pool_size=[2, 2], strides=2))
model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(10, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy, optimizer=keras.optimizers.Adam(),
              metrics=['accuracy'])

model.fit(x_train, y_train, batch_size=batch_size, epochs=n_epochs, verbose=1, validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])